In [1]:
# ============================================================
# v7_holdout_ensemble_b3_vit_convnext.py
# ------------------------------------------------------------
# - Train data: v7 (offline augmented)
#     BASE/meta_stage0_7_train_v7.csv (filepath, group, basename)
# - Raw labels: BASE/raw/train.csv (ID, target)
# - Test data: v6
#     BASE/processed/stage0_6_test_v6/, BASE/raw/sample_submission.csv
# - Strategy:
#     1) Holdout 8:2 (by original ID, stratified)
#     2) Train 3 models: EffB3, ViT-B16, ConvNeXt-S
#     3) Use all aug of train IDs for training
#     4) Holdout eval은 원본 이미지만 사용
#     5) 4-way TTA test inference: [0°,90°] x [flip,no-flip]
#     6) Holdout 기반 Temp scaling + weight grid search (w>=0.15)
#     7) Final ensemble submission
# ============================================================

import os, gc, re, math, json, random, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# ----------------------------
# Config
# ----------------------------
BASE = "/data/ephemeral/home/data"   # ✅ 여기만 네 환경에 맞게 조정

META_V7      = f"{BASE}/meta_stage0_7_train_v7.csv"
TRAIN_CSV    = f"{BASE}/raw/train.csv"
SUB_CSV      = f"{BASE}/raw/sample_submission.csv"
TRAIN_IMG_V7 = f"{BASE}/processed/stage0_7_train_v7"
TEST_IMG_V6  = f"{BASE}/processed/stage0_6_test_v6"

OUT_DIR      = "./runs_v7_holdout"
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(f"{OUT_DIR}/models", exist_ok=True)
os.makedirs(f"{OUT_DIR}/debug", exist_ok=True)
os.makedirs(f"{OUT_DIR}/test_probs", exist_ok=True)

CFG = dict(
    seed=42,
    num_classes=17,
    img_size_b3=380,
    img_size_vit=384,
    img_size_conv=384,
    batch_size=16,
    epochs=40,
    early_stop=12,
    lr_b3=1e-4,  wd_b3=1e-4,
    lr_vit=5e-5, wd_vit=0.05,
    lr_conv=2e-4, wd_conv=0.05,
    label_smooth=0.05,
    mixup_prob=0.5,
    mixup_alpha=0.2,
    device="cuda" if torch.cuda.is_available() else "cpu",
    tta_angles=[0, 90],
    tta_flips=[False, True],
)



In [2]:
# ============================================================
# Utils
# ============================================================
def set_seed(seed):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

def row_normalize(x):
    x = np.asarray(x, dtype=np.float64)
    x = np.clip(x, 1e-12, None)
    s = x.sum(1, keepdims=True)
    s[s == 0] = 1.0
    return (x / s).astype(np.float32)

def corrcoef(a, b):
    a = a.reshape(len(a), -1); b = b.reshape(len(b), -1)
    a = a - a.mean(axis=1, keepdims=True)
    b = b - b.mean(axis=1, keepdims=True)
    an = np.sqrt((a*a).sum(1, keepdims=True)) + 1e-12
    bn = np.sqrt((b*b).sum(1, keepdims=True)) + 1e-12
    a /= an; b /= bn
    return float(np.mean((a*b).sum(1)))

def temp_scale(probs, T):
    logits = np.log(np.clip(probs, 1e-12, 1.0))
    logits /= T
    m = logits.max(1, keepdims=True)
    e = np.exp(logits - m)
    return row_normalize(e)

def fit_temperature(probs, targets, t_min=1.0, t_max=2.0, steps=21):
    # simple grid-search NLL minimization
    ys = np.eye(CFG["num_classes"], dtype=np.float64)[targets]
    best_T, best_nll = 1.0, 1e18
    for T in np.linspace(t_min, t_max, steps):
        p = temp_scale(probs, T)
        nll = -np.sum(ys * np.log(np.clip(p,1e-12,1.0))) / len(targets)
        if nll < best_nll:
            best_nll, best_T = nll, T
    return float(best_T)

def save_npz(path, arr):
    arr = np.asarray(arr, dtype=np.float32)
    np.savez_compressed(path, arr)



In [3]:
# ============================================================
# Data prep: v7 + holdout split
# ============================================================
set_seed(CFG["seed"])

print("🔹 Load meta v7 & raw labels")
meta = pd.read_csv(META_V7)  # filepath, group, basename
train_raw = pd.read_csv(TRAIN_CSV)  # ID, target

# ID normalizer
def norm_id(x):
    x = str(x)
    return x if x.endswith(".jpg") else f"{x}.jpg"

train_raw["ID_norm"] = train_raw["ID"].apply(norm_id)
id2tgt = dict(zip(train_raw["ID_norm"], train_raw["target"]))

# root_id 추출 (aug 제거)
def get_root_id(basename):
    b = os.path.splitext(str(basename))[0]
    b = b.split("_aug")[0]
    return norm_id(b)

meta["root_id"] = meta["basename"].apply(get_root_id)
meta["target"] = meta["root_id"].map(id2tgt)

# 유효한 레코드만
meta = meta.dropna(subset=["target"]).reset_index(drop=True)
meta["target"] = meta["target"].astype(int)

# filepath 재구성 (meta의 filepath가 상대/절대 섞였을 수 있어서 신뢰 안 함)
meta["filepath"] = meta.apply(
    lambda r: os.path.join(TRAIN_IMG_V7, str(r["group"]), str(r["basename"])),
    axis=1
)

# 원본(비-aug) 기준으로 holdout split: 8:2 stratified
orig = meta[~meta["basename"].str.contains("_aug")][["root_id","target"]].drop_duplicates()
train_ids, holdout_ids = train_test_split(
    orig["root_id"],
    test_size=0.2,
    random_state=CFG["seed"],
    stratify=orig["target"]
)
train_ids = set(train_ids)
holdout_ids = set(holdout_ids)

# train: train_ids에 속한 모든 (원본+aug)
train_df = meta[meta["root_id"].isin(train_ids)].reset_index(drop=True)

# holdout: holdout_ids에 속한 "원본만" (aug 제외)
holdout_df = meta[
    (meta["root_id"].isin(holdout_ids)) &
    (~meta["basename"].str.contains("_aug"))
].reset_index(drop=True)

print(f"✅ Total v7: {len(meta)}")
print(f"✅ Train(root) IDs: {len(train_ids)}, Holdout(root) IDs: {len(holdout_ids)}")
print(f"✅ Train samples (incl. aug): {len(train_df)}")
print(f"✅ Holdout samples (orig only): {len(holdout_df)}")



🔹 Load meta v7 & raw labels
✅ Total v7: 7850
✅ Train(root) IDs: 1256, Holdout(root) IDs: 314
✅ Train samples (incl. aug): 6280
✅ Holdout samples (orig only): 314


In [4]:
# ============================================================
# Albumentations transforms
# ============================================================
def build_transforms(img_size, is_train=True):
    if is_train:
        return A.Compose([
            A.Resize(img_size, img_size),
            A.Rotate(limit=10, border_mode=cv2.BORDER_REFLECT_101, p=0.4),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(0.15, 0.15, p=0.4),
            A.GaussianBlur(blur_limit=(3,5), p=0.3),
            A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(img_size, img_size),
            A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
            ToTensorV2(),
        ])

# ============================================================
# Dataset
# ============================================================
class ImgDataset(Dataset):
    def __init__(self, df, img_size, is_train):
        self.df = df.reset_index(drop=True)
        self.tf = build_transforms(img_size, is_train)
        self.is_train = is_train
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread(row["filepath"])
        if img is None:
            raise FileNotFoundError(row["filepath"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        aug = self.tf(image=img)["image"]
        target = int(row["target"])
        return aug, target

class TestDataset(Dataset):
    def __init__(self, filepaths, img_size):
        self.paths = filepaths
        self.tf = build_transforms(img_size, is_train=False)
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img = cv2.imread(self.paths[idx])
        if img is None:
            raise FileNotFoundError(self.paths[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        aug = self.tf(image=img)["image"]
        return aug

# ============================================================
# Model factory
# ============================================================
def create_model(name, num_classes, drop_rate=0.0, drop_path=0.0):
    model = timm.create_model(
        name,
        pretrained=True,
        num_classes=num_classes,
        drop_rate=drop_rate,
        drop_path_rate=drop_path,
        in_chans=3,
    )
    model.to(CFG["device"])
    model.to(memory_format=torch.channels_last)
    if hasattr(model, "set_grad_checkpointing"):
        model.set_grad_checkpointing(True)
    return model

# ============================================================
# Train / Validate (holdout)
# ============================================================
def train_one_epoch(model, loader, optimizer, loss_fn, scaler):
    model.train()
    total_loss = 0.0
    for xb, yb in loader:
        xb = xb.to(CFG["device"], non_blocking=True)
        yb = yb.to(CFG["device"], non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(CFG["device"]=="cuda")):
            logits = model(xb)
            loss = loss_fn(logits, yb)
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
    return total_loss / max(1, len(loader))

@torch.no_grad()
def validate(model, loader, loss_fn):
    model.eval()
    total_loss = 0.0
    probs_all, targs_all = [], []
    for xb, yb in loader:
        xb = xb.to(CFG["device"], non_blocking=True)
        yb = yb.to(CFG["device"], non_blocking=True)
        with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(CFG["device"]=="cuda")):
            logits = model(xb)
            loss = loss_fn(logits, yb)
            probs = torch.softmax(logits, dim=1).to(torch.float32).cpu().numpy()
        total_loss += loss.item()
        probs_all.append(probs)
        targs_all.append(yb.cpu().numpy())
    probs_all = row_normalize(np.vstack(probs_all))
    targs_all = np.concatenate(targs_all)
    f1 = f1_score(targs_all, probs_all.argmax(1), average="macro")
    return total_loss / max(1, len(loader)), f1, probs_all, targs_all

def train_backbone(key, train_df, holdout_df):
    if key == "b3":
        name = "tf_efficientnet_b3_ns"
        img_size = CFG["img_size_b3"]
        lr, wd = CFG["lr_b3"], CFG["wd_b3"]
        drop_rate, drop_path = 0.4, 0.2
    elif key == "vit":
        name = "vit_base_patch16_384"
        img_size = CFG["img_size_vit"]
        lr, wd = CFG["lr_vit"], CFG["wd_vit"]
        drop_rate, drop_path = 0.2, 0.1
    elif key == "conv":
        name = "convnext_small"
        img_size = CFG["img_size_conv"]
        lr, wd = CFG["lr_conv"], CFG["wd_conv"]
        drop_rate, drop_path = 0.0, 0.2
    else:
        raise ValueError(key)

    print(f"\n==== Train {key} ({name}) ====")

    tr_ds = ImgDataset(train_df, img_size, is_train=True)
    ho_ds = ImgDataset(holdout_df, img_size, is_train=False)
    tr_ld = DataLoader(tr_ds, batch_size=CFG["batch_size"], shuffle=True,
                       num_workers=4, pin_memory=True, drop_last=True)
    ho_ld = DataLoader(ho_ds, batch_size=CFG["batch_size"], shuffle=False,
                       num_workers=4, pin_memory=True, drop_last=False)

    model = create_model(name, CFG["num_classes"], drop_rate, drop_path)
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.CrossEntropyLoss(label_smoothing=CFG["label_smooth"])
    scaler = torch.cuda.amp.GradScaler(enabled=(CFG["device"]=="cuda"))

    best_f1, best_probs, best_tgts = -1.0, None, None
    best_path = f"{OUT_DIR}/models/{key}_best.pt"
    es = 0

    for ep in range(1, CFG["epochs"]+1):
        tr_loss = train_one_epoch(model, tr_ld, optimizer, loss_fn, scaler)
        va_loss, va_f1, va_probs, va_tgts = validate(model, ho_ld, loss_fn)
        print(f"[{key}] Ep{ep}/{CFG['epochs']} | TrLoss={tr_loss:.4f} "
              f"VaLoss={va_loss:.4f} VaF1={va_f1:.4f}")

        if va_f1 > best_f1:
            best_f1 = va_f1
            best_probs, best_tgts = va_probs, va_tgts
            torch.save(model.state_dict(), best_path)
            es = 0
        else:
            es += 1
        if es >= CFG["early_stop"]:
            print(f"[{key}] Early stop at Ep{ep}")
            break

    print(f"✅ {key} best holdout F1 = {best_f1:.4f}")
    del model; torch.cuda.empty_cache(); gc.collect()
    return best_path, best_probs, best_tgts, best_f1



In [5]:
# ============================================================
# TTA inference
# ============================================================
def rotate_np(img, angle):
    if angle == 0: return img
    if angle == 90: return cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    raise ValueError("Only 0 and 90 used here.")

@torch.no_grad()
def infer_with_tta(key, model_path, filepaths):
    if key == "b3":
        name = "tf_efficientnet_b3_ns"
        img_size = CFG["img_size_b3"]
        drop_rate, drop_path = 0.4, 0.2
    elif key == "vit":
        name = "vit_base_patch16_384"
        img_size = CFG["img_size_vit"]
        drop_rate, drop_path = 0.2, 0.1
    elif key == "conv":
        name = "convnext_small"
        img_size = CFG["img_size_conv"]
        drop_rate, drop_path = 0.0, 0.2
    else:
        raise ValueError(key)

    tf = build_transforms(img_size, is_train=False)

    model = create_model(name, CFG["num_classes"], drop_rate, drop_path)
    state = torch.load(model_path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval()

    probs_total = None
    n_tta = len(CFG["tta_angles"]) * len(CFG["tta_flips"])

    for ang in CFG["tta_angles"]:
        for fl in CFG["tta_flips"]:
            batch_tensors = []
            for p in filepaths:
                img = cv2.imread(p)
                if img is None:
                    raise FileNotFoundError(p)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = rotate_np(img, ang)
                if fl:
                    img = cv2.flip(img, 1)
                t = tf(image=img)["image"]
                batch_tensors.append(t)
            ds = torch.stack(batch_tensors)
            ld = DataLoader(ds, batch_size=CFG["batch_size"], shuffle=False,
                            num_workers=2, pin_memory=True, drop_last=False)
            probs_run = []
            for xb in ld:
                xb = xb.to(CFG["device"], non_blocking=True)
                with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(CFG["device"]=="cuda")):
                    logits = model(xb)
                    pr = torch.softmax(logits, dim=1).to(torch.float32).cpu().numpy()
                probs_run.append(pr)
            probs_run = row_normalize(np.vstack(probs_run))
            probs_total = probs_run if probs_total is None else (probs_total + probs_run)

    del model; torch.cuda.empty_cache(); gc.collect()
    return row_normalize(probs_total / n_tta)



In [6]:
# ============================================================
# Weight search (3-model, each w>=0.15)
# ============================================================
def search_weights_3(p1, p2, p3, y, step=0.05, floor=0.15):
    best_f1, best_w = -1.0, (0.6,0.2,0.2)
    ws = np.arange(floor, 1.0 + 1e-9, step)
    for w1 in ws:
        for w2 in ws:
            w3 = 1.0 - w1 - w2
            if w3 < floor - 1e-9:
                continue
            mix = row_normalize(w1*p1 + w2*p2 + w3*p3)
            f1 = f1_score(y, mix.argmax(1), average="macro")
            if f1 > best_f1:
                best_f1, best_w = f1, (w1,w2,w3)
    return best_w, best_f1

# ============================================================
# Main
# ============================================================
if __name__ == "__main__":
    # 1) Train 3 backbones with holdout
    b3_path,  b3_ho_p,  ho_y, b3_f1  = train_backbone("b3",  train_df, holdout_df)
    vit_path, vit_ho_p, _,   vit_f1 = train_backbone("vit", train_df, holdout_df)
    conv_path,conv_ho_p,_,   conv_f1= train_backbone("conv",train_df, holdout_df)

    # 2) Temp scaling (per model) using holdout
    print("\n==== Fit temperature on holdout ====")
    T_b3  = fit_temperature(b3_ho_p,  ho_y, t_min=1.0, t_max=2.0)
    T_vit = fit_temperature(vit_ho_p, ho_y, t_min=1.0, t_max=2.0)
    T_conv= fit_temperature(conv_ho_p,ho_y, t_min=1.0, t_max=2.0)
    print(f"T_b3={T_b3:.2f}, T_vit={T_vit:.2f}, T_conv={T_conv:.2f}")

    b3_ho_c   = temp_scale(b3_ho_p,   T_b3)
    vit_ho_c  = temp_scale(vit_ho_p,  T_vit)
    conv_ho_c = temp_scale(conv_ho_p, T_conv)

    # 3) Weight search on holdout (calibrated)
    print("\n==== Search ensemble weights on holdout ====")
    (w_b3, w_vit, w_conv), best_ho_f1 = search_weights_3(
        b3_ho_c, vit_ho_c, conv_ho_c, ho_y,
        step=0.05, floor=0.15
    )
    print(f"Best Holdout F1={best_ho_f1:.4f} | weights: "
          f"b3={w_b3:.2f}, vit={w_vit:.2f}, conv={w_conv:.2f}")

    # 4) Test paths aligned with sample_submission
    sub = pd.read_csv(SUB_CSV)
    sub_ids = sub["ID"].astype(str).apply(lambda x: x if x.endswith(".jpg") else f"{x}.jpg")
    test_files = []
    for name in sub_ids:
        cands = []
        # allow nested dirs just in case
        for root, _, files in os.walk(TEST_IMG_V6):
            if name in files:
                cands = [os.path.join(root, name)]
                break
        if cands:
            test_files.append(cands[0])
        else:
            test_files.append(os.path.join(TEST_IMG_V6, name))

    print(f"\n==== TTA inference on test (v6) ====")
    b3_test   = infer_with_tta("b3",   b3_path,   test_files)
    vit_test  = infer_with_tta("vit",  vit_path,  test_files)
    conv_test = infer_with_tta("conv", conv_path, test_files)

    save_npz(f"{OUT_DIR}/test_probs/b3_test_probs.npz",   b3_test)
    save_npz(f"{OUT_DIR}/test_probs/vit_test_probs.npz",  vit_test)
    save_npz(f"{OUT_DIR}/test_probs/conv_test_probs.npz", conv_test)

    # 5) Apply same temperature scaling to test probs
    b3_test_c   = temp_scale(b3_test,   T_b3)
    vit_test_c  = temp_scale(vit_test,  T_vit)
    conv_test_c = temp_scale(conv_test, T_conv)

    # 6) Final ensemble
    mix = row_normalize(
        w_b3*b3_test_c + w_vit*vit_test_c + w_conv*conv_test_c
    )
    preds = mix.argmax(1)

    # 7) Diagnostics
    with open(f"{OUT_DIR}/debug/ensemble_diagnostics.txt", "w") as f:
        f.write(f"Holdout best F1={best_ho_f1:.4f}\n")
        f.write(f"Weights: b3={w_b3:.3f}, vit={w_vit:.3f}, conv={w_conv:.3f}\n")
        f.write(f"T: b3={T_b3:.3f}, vit={T_vit:.3f}, conv={T_conv:.3f}\n")
        f.write(f"Corr(Eff,ViT)={corrcoef(b3_test, vit_test):.4f}\n")
        f.write(f"Corr(Eff,Conv)={corrcoef(b3_test, conv_test):.4f}\n")
        f.write(f"Corr(ViT,Conv)={corrcoef(vit_test, conv_test):.4f}\n")
        f.write(f"RowSumDelta={np.mean(np.abs(mix.sum(1)-1.0)):.3e}\n")




==== Train b3 (tf_efficientnet_b3_ns) ====
[b3] Ep1/40 | TrLoss=1.2527 VaLoss=0.6379 VaF1=0.8609
[b3] Ep2/40 | TrLoss=0.6129 VaLoss=0.5625 VaF1=0.9066
[b3] Ep3/40 | TrLoss=0.4942 VaLoss=0.5216 VaF1=0.9364
[b3] Ep4/40 | TrLoss=0.4501 VaLoss=0.5185 VaF1=0.9298
[b3] Ep5/40 | TrLoss=0.4171 VaLoss=0.4921 VaF1=0.9455
[b3] Ep6/40 | TrLoss=0.4004 VaLoss=0.5077 VaF1=0.9392
[b3] Ep7/40 | TrLoss=0.3865 VaLoss=0.5232 VaF1=0.9377
[b3] Ep8/40 | TrLoss=0.3742 VaLoss=0.5068 VaF1=0.9440
[b3] Ep9/40 | TrLoss=0.3705 VaLoss=0.4942 VaF1=0.9389
[b3] Ep10/40 | TrLoss=0.3627 VaLoss=0.4908 VaF1=0.9351
[b3] Ep11/40 | TrLoss=0.3581 VaLoss=0.4931 VaF1=0.9511
[b3] Ep12/40 | TrLoss=0.3550 VaLoss=0.4913 VaF1=0.9416
[b3] Ep13/40 | TrLoss=0.3586 VaLoss=0.5278 VaF1=0.9377
[b3] Ep14/40 | TrLoss=0.3533 VaLoss=0.5025 VaF1=0.9405
[b3] Ep15/40 | TrLoss=0.3478 VaLoss=0.5020 VaF1=0.9337
[b3] Ep16/40 | TrLoss=0.3444 VaLoss=0.5016 VaF1=0.9262
[b3] Ep17/40 | TrLoss=0.3490 VaLoss=0.4863 VaF1=0.9394
[b3] Ep18/40 | TrLoss=0.3490 V

model.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

[conv] Ep1/40 | TrLoss=2.7335 VaLoss=2.3269 VaF1=0.1022
[conv] Ep2/40 | TrLoss=2.1522 VaLoss=1.8299 VaF1=0.3153
[conv] Ep3/40 | TrLoss=1.6627 VaLoss=1.2460 VaF1=0.5300
[conv] Ep4/40 | TrLoss=1.1912 VaLoss=0.9211 VaF1=0.7424
[conv] Ep5/40 | TrLoss=0.9289 VaLoss=0.8435 VaF1=0.7475
[conv] Ep6/40 | TrLoss=0.8064 VaLoss=0.7079 VaF1=0.8236
[conv] Ep7/40 | TrLoss=0.6974 VaLoss=0.6877 VaF1=0.8377
[conv] Ep8/40 | TrLoss=0.6280 VaLoss=0.6226 VaF1=0.8709
[conv] Ep9/40 | TrLoss=0.5825 VaLoss=0.6077 VaF1=0.8880
[conv] Ep10/40 | TrLoss=0.5270 VaLoss=0.5754 VaF1=0.9128
[conv] Ep11/40 | TrLoss=0.5143 VaLoss=0.6255 VaF1=0.8904
[conv] Ep12/40 | TrLoss=0.4734 VaLoss=0.5776 VaF1=0.9074
[conv] Ep13/40 | TrLoss=0.4686 VaLoss=0.6271 VaF1=0.9041
[conv] Ep14/40 | TrLoss=0.4492 VaLoss=0.6188 VaF1=0.8927
[conv] Ep15/40 | TrLoss=0.4132 VaLoss=0.6832 VaF1=0.8860
[conv] Ep16/40 | TrLoss=0.4002 VaLoss=0.6139 VaF1=0.8961
[conv] Ep17/40 | TrLoss=0.4235 VaLoss=0.6421 VaF1=0.8910
[conv] Ep18/40 | TrLoss=0.4300 VaLoss=0.

In [7]:
    # 8) Save submission
    out_name = f"{OUT_DIR}/submission_ensemble_F{best_ho_f1:.4f}_w{w_b3:.2f}-{w_vit:.2f}-{w_conv:.2f}.csv"
    sub_out = pd.DataFrame({"ID": sub_ids, "target": preds})
    sub_out.to_csv(out_name, index=False)
    print(f"\n✅ Saved submission: {out_name}")



✅ Saved submission: ./runs_v7_holdout/submission_ensemble_F0.9565_w0.70-0.15-0.15.csv


In [9]:
import numpy as np
import os

OUT_DIR = "./runs_v7_holdout/test_probs"

for key in ["b3", "vit", "conv"]:
    path = os.path.join(OUT_DIR, f"{key}_test_probs.npz")
    if not os.path.exists(path):
        print(f"⚠️ {key}_test_probs.npz not found")
        continue
    arr = np.load(path)["arr_0"]
    delta = np.mean(np.abs(arr.sum(1) - 1.0))
    print(f"[{key.upper()}_TTA] shape={arr.shape}, rowSumMeanDelta={delta:.2e}")


[B3_TTA] shape=(3140, 17), rowSumMeanDelta=1.96e-08
[VIT_TTA] shape=(3140, 17), rowSumMeanDelta=2.12e-08
[CONV_TTA] shape=(3140, 17), rowSumMeanDelta=2.08e-08
